In [27]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

In [28]:
df = pd.read_csv("dataset/201306-citibike-tripdata.csv")
df.shape

(577703, 15)

In [29]:
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,695,2013-06-01 00:00:01,2013-06-01 00:11:36,444,Broadway & W 24 St,40.742354,-73.989151,434.0,9 Ave & W 18 St,40.743174,-74.003664,19678,Subscriber,1983.0,1
1,693,2013-06-01 00:00:08,2013-06-01 00:11:41,444,Broadway & W 24 St,40.742354,-73.989151,434.0,9 Ave & W 18 St,40.743174,-74.003664,16649,Subscriber,1984.0,1
2,2059,2013-06-01 00:00:44,2013-06-01 00:35:03,406,Hicks St & Montague St,40.695128,-73.995951,406.0,Hicks St & Montague St,40.695128,-73.995951,19599,Customer,NaN,0
3,123,2013-06-01 00:01:04,2013-06-01 00:03:07,475,E 15 St & Irving Pl,40.735243,-73.987586,262.0,Washington Park,40.691782,-73.973730,16352,Subscriber,1960.0,1
4,1521,2013-06-01 00:01:22,2013-06-01 00:26:43,2008,Little West St & 1 Pl,40.705693,-74.016777,310.0,State St & Smith St,40.689269,-73.989129,15567,Subscriber,1983.0,1


In [30]:
df.describe()

,tripduration,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,bikeid,birth year,gender
count,5.777030e+05,577703.000000,577703.000000,577703.000000,559644.000000,559644.000000,559644.000000,577703.000000,337382.000000,577703.000000
mean,1.372570e+03,439.228836,40.732436,-73.991251,441.910624,40.732146,-73.991244,17617.434516,1974.644945,0.710431
std,9.028063e+03,337.240318,0.021423,0.013002,345.448311,0.021507,0.013105,1677.518719,10.780255,0.678268
min,6.100000e+01,72.000000,40.646607,-74.017134,72.000000,40.646607,-74.017134,14529.000000,1899.000000,0.000000
25%,5.140000e+02,309.000000,40.717400,-74.001768,309.000000,40.716887,-74.001971,16161.000000,1968.000000,0.000000
50%,8.740000e+02,404.000000,40.734546,-73.990741,403.000000,40.734232,-73.990741,17638.000000,1977.000000,1.000000
75%,1.398000e+03,482.000000,40.749013,-73.981948,482.000000,40.748549,-73.981923,19066.000000,1983.000000,1.000000
max,3.876479e+06,3020.000000,40.771522,-73.950048,3020.000000,40.771522,-73.950048,20625.000000,1997.000000,2.000000


##### Checking for trips less than a specific time(<90 seconds)

In [31]:
# Checking for trip duration
broken_rides_df = df[df['tripduration'] <= 90]

# Checking for same end station
broken_rides_df = broken_rides_df[broken_rides_df['start station name'] == broken_rides_df['end station name']]

broken_rides_count = broken_rides_df.shape[0]
print("Number of broken rides:", broken_rides_count)

Number of broken rides: 2040


In [32]:
min_duration = 90  # Minimum acceptable trip duration (seconds)

broken_rides_mask = (df['tripduration'] <= min_duration) & (df['start station name'] == df['end station name'])
filtered_df = df[~broken_rides_mask] 

print("Original DataFrame size:", df.shape[0])
print("Filtered DataFrame size:", filtered_df.shape[0])


Original DataFrame size: 577703
Filtered DataFrame size: 575663


In [33]:
df.isnull().sum()

tripduration                    0
starttime                       0
stoptime                        0
start station id                0
start station name              0
start station latitude          0
start station longitude         0
end station id              18059
end station name            18059
end station latitude        18059
end station longitude       18059
bikeid                          0
usertype                        0
birth year                 240321
gender                          0
dtype: int64

##### Here we see null entries on end station in some of the entries. 

In [34]:
completed_rides_df = filtered_df.dropna()

completed_rides_df.shape

(325466, 15)

##### Now, completed_rides_df contains only the entries that are not present in broken_rides_df

## 1. Which 2 stations are most used by riders.

In [35]:
# Group by start station and sort by count in descending order
station_counts = df.groupby('start station name').size().sort_values(ascending=False)

# Getting the top 2 stations
top_2_stations = station_counts.head(2)

# Print the station names and their respective counts
print("Top 2 most used stations:")
for station, count in top_2_stations.items():
    print(f"- {station}: {count} initiated rides")

Top 2 most used stations:
- W 20 St & 11 Ave: 5983 initiated rides
- E 17 St & Broadway: 5621 initiated rides


In [36]:
# Group by start station for completed rides only and sort by count in descending order
station_counts = completed_rides_df.groupby('start station name').size().sort_values(ascending=False)

# Get the top 2 stations
top_2_stations = station_counts.head(2)

# Print the station names and their respective counts
print("Top 2 most used stations:")
for station, count in top_2_stations.items():
    print(f"- {station}: {count} for completed rides")

Top 2 most used stations:
- E 17 St & Broadway: 3351 for completed rides
- Broadway & E 14 St: 3108 for completed rides


##### W 20 St & 11 Ave have the most bikes stolen

## 2. Which month is the busiest time for Citi bike?


In [37]:
# Ensure 'starttime' is datetime-formatted
if not pd.api.types.is_datetime64_dtype(filtered_df['starttime']):
    filtered_df['starttime'] = pd.to_datetime(filtered_df['starttime'], format='%Y-%m-%d %H:%M:%S')

# Extract and store month name
filtered_df['month'] = filtered_df['starttime'].dt.month_name()

# 3. Group data and calculate total durations
monthly_durations = filtered_df.groupby('month')['tripduration'].sum()

# 4. Identify and print busiest month with comprehensive information
if not monthly_durations.empty:
    busiest_month = monthly_durations.idxmax()
    busiest_month_duration = monthly_durations.max() / 3600

    print(f"Busiest month: {busiest_month}")
    print(f"Total ride duration in {busiest_month}: {busiest_month_duration:.2f} hours")
else:
    print("No rides found in the DataFrame.")


Busiest month: June
Total ride duration in June: 220218.44 hours



### 3. Whats the average time of ride?


In [38]:
average_duration = completed_rides_df['tripduration'].mean()

# Convert seconds to minutes
average_minutes = average_duration / 60

print(f"Average ride time: {average_minutes:.2f} minutes")

Average ride time: 18.38 minutes


### 4. How many rides had same origin and destination?

In [39]:
same_origin_destination = filtered_df[filtered_df['start station name'] == filtered_df['end station name']]
count_same_origin_destination = same_origin_destination.shape[0]

print(f"Number of rides with same origin and destination: {count_same_origin_destination}")


Number of rides with same origin and destination: 21584
